# **Assignment 1 (10 points)**

## **Solving a linear system in python**

In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.

We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:
$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad 
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad 
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**
- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms

### **1. Parsing the System of Equations (1 point)**

The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**
```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated

In [57]:
import pathlib
import re

def process_line(line: str) -> tuple[list[float], float]:
    lhs, rhs = line.split('=')
    b_value = float(rhs.strip())

    coefficients = re.findall(r'([+-]?)\s*(\d*\.?\d*)?\s*([xyz])', lhs)
    print(coefficients)

    a_coeff = [0.0, 0.0, 0.0]

    for sign, coeff, unknown in coefficients:
        if coeff == '':
            coeff = 1.0
        else:
            coeff = float(coeff)

        if sign == '-':
            coeff *= -1.0

        if unknown == 'x':
            a_coeff[0] += coeff
        elif unknown == 'y':
            a_coeff[1] += coeff
        elif unknown == 'z':
            a_coeff[2] += coeff

    return a_coeff, b_value
    
def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    A = []
    B = []

    # Read the file and process each line
    file_contents = path.read_text().strip()
    for line in file_contents.split('\n'):
        coefficients, b_value = process_line(line)
        A.append(coefficients)
        B.append(b_value)

    return A, B

# Example usage
A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")

[('', '2', 'x'), ('+', '3', 'y'), ('-', '', 'z')]
[('', '', 'x'), ('-', '', 'y'), ('+', '4', 'z')]
[('', '3', 'x'), ('+', '', 'y'), ('+', '2', 'z')]
A=[[2.0, 3.0, -1.0], [1.0, -1.0, 4.0], [3.0, 1.0, 2.0]] B=[5.0, 6.0, 7.0]


### **2. Matrix and Vector Operations (5 points)**

Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:

#### 2.1. Determinant

Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:|
$$ \text{det}(A) = a_{11}(a_{22}a_{33} - a_{23}a_{32}) - a_{12}(a_{21}a_{33} - a_{23}a_{31}) + a_{13}(a_{21}a_{32} - a_{22}a_{31}) $$

In [73]:
def determinant(matrix: list[list[float]]) -> float:
    n = len(matrix)
    if n == 1:
        return matrix[0][0]
    elif n == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]

    det = 0
    for c in range(n):
        minor_matrix = [row[:c] + row[c+1:] for row in matrix[1:]]
        # Recursive call to calculate the determinant of the minor
        det += ((-1) ** c) * matrix[0][c] * determinant(minor_matrix)

    return det

#### 2.2. Trace

Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a_{11} + a_{22} + a_{33} $$

In [62]:
def trace(matrix: list[list[float]]) -> float:
    return sum([matrix[i][i] for i in range(len(matrix))])

print(f"{trace(A)=}")

trace(A)=3.0


#### 2.3. Vector norm

Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$

In [67]:
import math

def norm(vector: list[float]) -> float:
    return math.sqrt(sum([b*b for b in vector]))

print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.
    

In [68]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]

print(f"{transpose(A)=}")

transpose(A)=[[2.0, 1.0, 3.0], [3.0, -1.0, 1.0], [-1.0, 4.0, 2.0]]


#### 2.5. Matrix-vector multiplication

Write a function that multiplies matrix $A$ with vector $B$.

In [70]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
    result = [0] * len(matrix)
    for i in range(len(matrix)):
        result[i] = sum(matrix[i][j] * vector[j] for j in range(len(vector)))

    return result

print(f"{multiply(A, B)=}")

multiply(A, B)=[21.0, 27.0, 35.0]


### **3. Solving using Cramer's Rule (1 point)**

Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.

In [71]:
def replace_column(matrix: list[list[float]], vector: list[float], col_index: int) -> list[list[float]]:
    new_matrix = [row[:] for row in matrix]
    for i in range(len(matrix)):
        new_matrix[i][col_index] = vector[i]
    return new_matrix

def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
    D = determinant(matrix)
    if D == 0:
        raise ValueError("det=0 => no solution")

    D_x = determinant(replace_column(matrix, vector, 0))  # Replace first column
    D_y = determinant(replace_column(matrix, vector, 1))  # Replace second column
    D_z = determinant(replace_column(matrix, vector, 2))  # Replace third column

    x = D_x / D
    y = D_y / D
    z = D_z / D
    
    return [x, y, z]
print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**

Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M_{ij}) $$
where $M_{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.

In [74]:
def minor(matrix: list[list[float]], i: int, j: int) -> float:
    submatrix = [[matrix[x][y] for y in range(3) if y != j] for x in range(3) if x != i]
    return determinant(submatrix)

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
    cofactors = []
    for i in range(3):
        row = []
        for j in range(3):
            sign = (-1) ** (i + j)
            row.append(sign * minor(matrix, i, j)) 
        cofactors.append(row)
    return cofactors

def adjoint(matrix: list[list[float]]) -> list[list[float]]:
    cofactors = cofactor(matrix)
    # Transpose the cofactor matrix
    return [[cofactors[j][i] for j in range(3)] for i in range(3)]

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
    det = determinant(matrix)
    if det == 0:
        raise ValueError("det=0 => no solution")

    adj = adjoint(matrix)  

    solution = []
    for i in range(3):
        value = sum(adj[i][j] * vector[j] for j in range(3)) / det
        solution.append(value)

    return solution

print(f"{solve(A, B)=}")

solve(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]
